## Import libraries needed

In [4]:
import pandas as pd
import numpy as np
import boto3
from pyathena import connect
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
## set the session and roles for AWS
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Setup connection to Athena to pull in tables

In [6]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [8]:
# create the connection, this will be used anytime we need to query data from Athena
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [14]:
## Verify we have the database created
statement = "SHOW DATABASES"
db_show = pd.read_sql(statement, conn)
db_show.head(5)
## store teh db_name in a variable
db_name = str(db_show.iloc[0]['database_name'])

In [29]:
## we need to query the tables in and do a Join so it is one table to run our EDA off
db_name

'ads_508_team_5'

In [40]:
join_statement = """SELECT * FROM
                    ads_508_team_5.customers_dataset c
                    LEFT JOIN ads_508_team_5.orders_dataset o on c.customer_id = o.customer_id
                    LEFT JOIN ads_508_team_5.order_payments_dataset op on o.order_id = op.order_id
                    LEFT JOIN ads_508_team_5.order_reviews_dataset ord on op.order_id = ord.order_id
                    LEFT JOIN ads_508_team_5.order_items_dataset oid on ord.order_id = oid.order_id
                    LEFT JOIN ads_508_team_5.products_dataset pd on oid.product_id = pd.product_id
                    LEFT JOIN ads_508_team_5.sellers_dataset sd on oid.seller_id = sd.seller_id
                    LEFT JOIN ads_508_team_5.geolocation_dataset gd_c on c.customer_zip_code_prefix = gd_c.geolocation_zip_code_prefix
                    LEFT JOIN ads_508_team_5.geolocation_dataset gd_s ON sd.seller_zip_code_prefix = gd_s.geolocation_zip_code_prefix
                    """

In [ ]:
main_df = pd.read_sql(join_statement, conn)
main_df.head()

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}